# Assignment 1, Extragalactic Astronomy
Nasser Mohammed

Feburary 3rd, 2025

> The purpose of this question is to familiarize you with the basics of luminosity functions and with the relationship between the luminosity function, luminosity density, and mass density.
> Assume that galaxies can be described by a Schechter function luminosity function given in Table 4 of Loveday et al. 2015, MNRAS, 451, 2, 1540-1552.

## 1.

> Write some code to compute the local number density of blue and red galaxies in the absolute magnitude range -23 < Mr < -14. Supply both the code and the answer. Any software or language is fine, though all things being equal I recommend a Jupyter notebook with Python for simplicity. In any case, I will zing you mercilessly if you don’t comment your notebook (or code file) appropriately to make it clear what you are doing.

In [1]:
import numpy as numpy
import matplotlib.pyplot as plt